**1. Créer un compte développeur sur https://developer.spotify.com**

**2. Créer une app**

**3. Récupérer le Client ID et le Client Secret et les stocker dans un fichier .env**

In [ ]:
SPOTIFY_CLIENT_ID=********
SPOTIFY_CLIENT_SECRET=********

**4. Dans un fichier Python, utiliser python-dotenv pour récupérer ces credentials :**

In [ ]:
import os

from dotenv import load_dotenv


load_dotenv()

client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

**5. Créer une fonction get_token qui gère l'authentification et la récupération du token nécessaire à l'utilisation de l'API**

In [ ]:
import base64
import json

from requests import post


def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

token = get_token()
print(token)

**6. Créer une fonction get_auth_headers qui génère le header à passer lors de nos requêtes à l'API**

In [ ]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

**7. Créer une fonction search_for_artist pour récupérer l'ID de l'artiste recherché**

In [ ]:
from requests import get


def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist found")
        return None
    return json_result[0]

result = search_for_artist(token, "ACDC")
artist_id = result["id"]
print(artist_id)

**8. Créer une fonction get_songs_by_artist pour récupérer la liste des 10 chansons les plus populaires d'un artiste**

In [ ]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"    
    headers = get_auth_header(token)

    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

songs = get_songs_by_artist(token, artist_id)

for idx, song in enumerate(songs):
    print(f"{idx + 1}. {song['name']}")